In [1]:
from numpy import *
import numpy as np
import pandas as pd
import keras
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv3D, MaxPooling3D
BATCH_SIZE = 50
NUM_CLASSES = 2
NUM_EPOCHS = 20

Using TensorFlow backend.


In [2]:
adress1='C:/Users/29276/Desktop/machinelearnling/train_val'
adress2='C:/Users/29276/Desktop/machinelearnling'
adress3='C:/Users/29276/Desktop/machinelearnling/test'
adress4='C:/Users/29276/Desktop/machinelearnling'

In [3]:
import os
x1 = np.load(adress1+'/candidate'+str(1)+'.npz')
x_train=[x1 for p in range(465)]
x_trainvoxel=[x1['voxel'] for p in range(465)]
x_trainseg=[x1['seg'] for p in range(465)]
j=0
i=0
while (i<584) :
    if(os.path.exists(adress1+'/candidate'+str(i)+'.npz')):
        x1 = np.load(adress1+'/candidate'+str(i)+'.npz')
        x_train[j]=x1
        x_trainvoxel[j]=x_train[j]['voxel']
        x_trainseg[j]=x_train[j]['seg']
        j+=1
    i+=1
x_train=x_trainvoxel*(np.array(x_trainseg).astype(int))
np.shape(x_train)

(465, 100, 100, 100)

In [4]:
y_train1=pd.read_csv(adress2+'/train_val.csv')
y_train=[1 for p in range(465)]
for z in range(265):
    y_train[z]=y_train1['lable'][z]
y_train1=y_train

In [5]:
import os
x2 = np.load(adress3+'/candidate'+str(11)+'.npz')
x_test=[x2 for p in range(117)]
x_testvoxel=[x2['voxel'] for p in range(117)]
x_testseg=[x2['seg'] for p in range(117)]
y_testname=[' ' for p in range(117)]
j=0
i=0
while (i<583) :
    if(os.path.exists(adress3+'/candidate'+str(i)+'.npz')):
        x2 = np.load(adress3+'/candidate'+str(i)+'.npz')
        x_test[j]=x2
        x_testvoxel[j]=x_test[j]['voxel']
        x_testseg[j]=x_test[j]['seg']
        y_testname[j]='candidate'+str(i)
        j+=1
    i+=1
x_test=x_testvoxel*(np.array(x_testseg).astype(int))
np.shape(x_test)


(117, 100, 100, 100)

In [6]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train2 = x_train[:,30:70,30:70,30:70]
x_test2 = x_test[:,30:70,30:70,30:70]
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
#数据增强
x_train3=x_train2
for h in range(0,462,7):
    #平移
    x_train3[h,0:39,:,:]=x_train2[h,1:40,:,:]
    x_train3[h,39,:,:]=x_train2[h,0,:,:]
    
    x_train3[h+1,1:40,:,:]=x_train2[h+1,0:39,:,:]
    x_train3[h+1,0,:,:]=x_train2[h+1,39,:,:]
    
    x_train3[h+2,:,0:39,:]=x_train2[h+2,:,1:40,:]
    x_train3[h+2,:,39,:]=x_train2[h+2,:,0,:]
    
    x_train3[h+3,:,1:40,:]=x_train2[h+3,:,0:39,:]
    x_train3[h+3,:,0,:]=x_train2[h+3,:,39,:]
    
    x_train3[h+4,:,:,0:39]=x_train2[h+4,:,:,1:40]
    x_train3[h+4,:,:,39]=x_train2[h+4,:,:,0]
    
    x_train3[h+5,:,:,1:40]=x_train2[h+5,:,:,0:39]
    x_train3[h+5,:,:,0]=x_train2[h+5,:,:,39]
    
    noise=np.random.normal(loc=0.0, scale=0.1, size=(40,40,40))
    x_train3[h+6,:,:,:]=x_train2[h+6,:,:,:]+ noise
    
#    for q in range(40):
#        x_train3[h+7,q,:,:]=x_train2[h+7,39-q,:,:]
#    for qq in range(40):
#        x_train3[h+8,:,qq,:]=x_train2[h+8,:,39-qq,:]

x_train4=np.concatenate((x_train2, x_train3), axis = 0)
y_train2=np.concatenate((y_train,y_train), axis = 0)

x_train2 = np.reshape(x_train2,(465,40,40,40,1))
x_train4 = np.reshape(x_train4,(930,40,40,40,1))
x_test2 = np.reshape(x_test2,(117,40,40,40,1))
#input_shape = (100, 100, 100)


In [7]:
model=keras.models.load_model(adress4+'/my_model.h5')

In [8]:
a=model.predict(x_test2)
b=a[:,1]
#c=[0 for p in range(117)]
#a10=[a for p in range(10)]
#b10=[b for p in range(10)]
#for h in range(10):
#    a10[h]=model.predict(x_test2)
#    b10[h]=a10[h][:,1]
#    c=c+b10[h]
#c=c/10
#dataframe = pd.DataFrame({'Id':y_testname,'Predicted':c})
dataframe = pd.DataFrame({'Id':y_testname,'Predicted':b})
dataframe.to_csv(adress2+'/Submission.csv',index=False,sep=',')

In [9]:
from sklearn.metrics import roc_auc_score
c=model.predict(x_train2)
d=[0 for p in range(465)]
for z in range(465):
    if y_train1[z]==0:
        d[z]=c[z][0]
    else:
        d[z]=c[z][1]
roc_auc_score(y_train1,d)

0.7716763005780347